Продолжение препроцессинга данных

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import pymorphy3
from tqdm.auto import tqdm
from collections import Counter
import razdel
import seaborn as sns
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
# nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

Конфигурационные переменные

In [64]:
DATASET_PATH = "../data/raw/labeled.csv"
SAVE_PROCESSED_PATH = "../data/processed/raw_features_extracted.csv"
GENERATED_PATH = "../data/processed/generated_agressive.csv"
PORNO_DATASET_PATH = "../data/raw/dataset_spot.xlsx"
SEED = 42
np.random.seed(SEED)

TOKEN_PATTERN = "[а-яёa-z]+"
SIGN_PATTERN = r'[!?*@#$&%^:;"\')(~`0-9]'
URL_PATTERN = r'https?://\S+|www\.\S+'
REPEATED_PATTERN = r'(.)\1{2,}'
PRONOUNS = ['я', 'ты', 'вы', 'он', 'она', 'оно', 'мы', 'они', 'вас', 'нас', 'их', 'его', 'её']


Считываем обновленный датасет

In [65]:
def read_dataset(data_path: str = SAVE_PROCESSED_PATH):
    dataset = pd.read_csv(data_path).drop(columns=['Unnamed: 0'])
    return dataset

comments_dataset = read_dataset()
comments_dataset.head()

,comment,toxic,text_length,potential_agressive_symbols,num_words,link_count,repeated_chars_count,pronoun_count,avg_word_length,caps_symbols
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,35,1,5,0,1,0,6.000000,2
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,121,0,21,0,0,1,4.761905,2
2,Собаке - собачья смерть\n,1.0,24,0,4,0,0,0,5.000000,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,184,1,32,0,0,2,4.750000,3
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,120,5,18,0,0,0,5.666667,4


Отсавим от него только две колонки

In [66]:
comments_dataset = comments_dataset.iloc[:, [0, 1]]
comments_dataset.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


Добавим к датасету несколько сэмплов из нового датасета на тему 18+

In [67]:
porno_dataset = pd.read_excel(PORNO_DATASET_PATH).iloc[:, [2, 3]]
porno_dataset.head()

,title,label
0,экс министр экономика молдова глава мидэи цель...,0
1,песня стать известный многий телезритель благо...,0
2,банши сезон серия бремя красота смотреть онлайн,0
3,новомосковск сыграть следж хоккеист алексински...,0
4,салат корейский морковь копчёный курица кукуру...,0


In [68]:
agressive_dataset = pd.read_csv(GENERATED_PATH).drop(columns = ['Unnamed: 0'])
agressive_dataset.head()

,comment,toxic
0,Почему никто не может сделать нормальный детск...,1
1,"Ты хоть представляешь, как меня бесит эта твоя...",1
2,"Ты когда-нибудь задумывался, какой ужас творит...",1
3,"Почему каждый раз, когда мне нужно купить комм...",1
4,"Ты что, совсем охренел? Какого фаллос черта ты...",1


In [69]:
comments_dataset = pd.concat([comments_dataset, agressive_dataset], axis = 0)

In [77]:
comments_dataset.shape

(15104, 2)

Корпус текстов

In [78]:
corpus_texts = comments_dataset.comment.values
corpus_texts[:10]

array(['Верблюдов-то за что? Дебилы, бл...\n',
       'Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n',
       'Собаке - собачья смерть\n',
       'Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n',
       'тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n',
       'Для каких стан является эталоном современная система здравоохранения РФ? Для Зимбабве? Ты тупой? хохлы\n',
       'В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.\n',
       'УПАД Т! ТАМ НЕЛЬЗЯ СТРОИ

Токенизация

In [79]:
def tokenize(text):
    return re.findall(TOKEN_PATTERN, text.lower())

documents = [tokenize(str(text)) for text in corpus_texts]

Число слов

In [80]:
occurence = Counter([text for document in documents for text in document])
len(occurence)

71861

In [81]:
occurence.most_common(10)

[('и', 16480),
 ('в', 13042),
 ('не', 12590),
 ('на', 8294),
 ('что', 7986),
 ('это', 6624),
 ('а', 5616),
 ('с', 4720),
 ('то', 4542),
 ('я', 4436)]

In [82]:
stopword_set = set(nltk.corpus.stopwords.words('russian'))
stopword_set_english = set(nltk.corpus.stopwords.words('english'))
# stopword_set = stopword_set.union({'это', 'который', 'весь', 'наш', 'свой', 'ещё', 'её', 'ваш', 'также', 'итак'})
stopword_set = stopword_set.union(stopword_set_english)

Очищаем корпусы

In [83]:
lemmatizer = pymorphy3.MorphAnalyzer()

lemmatizer_cache = {}

def lemmatize(token):
    if lemmatizer.word_is_known(token):
        if token not in lemmatizer_cache:
            lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
        return lemmatizer_cache[token]
    return token

lemmatized_docs = [[lemmatize(token) for token in text] for text in tqdm(documents)]

cleared_docs = [[token for token in text if token not in stopword_set] for text in lemmatized_docs]

  0%|          | 0/15104 [00:00<?, ?it/s]

In [84]:
len({token for text in cleared_docs for token in text})

37111

In [85]:
comments_dataset['cleared_text'] = cleared_docs

In [86]:
def len_zero(x):
    return len(x) != 0

comments_dataset['length_zero'] = comments_dataset[['cleared_text']].applymap(len_zero)
comments_dataset = comments_dataset.drop(columns = ['length_zero'])

In [87]:
def text_regenerator(x):
    return ' '.join(x)

comments_dataset['cleared_text'] = comments_dataset[['cleared_text']].applymap(text_regenerator)

In [88]:
comments_dataset.head()

,comment,toxic,cleared_text
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,верблюд дебил бл
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,хохол это отдушина затюканого россиянин мол во...
2,Собаке - собачья смерть\n,1.0,собака собачий смерть
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,страница обновить дебил это оскорбление доказа...
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,убедить страничный пдф скрипалей отравить росс...


Попробуем построить Word2Vec для классификации

In [89]:
def splitter(x):
    return x.split()

comments_dataset['cleared_text'] = comments_dataset[['cleared_text']].applymap(splitter)

Итак, датасет имеет вид:

In [90]:
comments_dataset = comments_dataset.iloc[:, [0, 1, 2]]
comments_dataset.head()

,comment,toxic,cleared_text
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"[верблюд, дебил, бл]"
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"[хохол, это, отдушина, затюканого, россиянин, ..."
2,Собаке - собачья смерть\n,1.0,"[собака, собачий, смерть]"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"[страница, обновить, дебил, это, оскорбление, ..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"[убедить, страничный, пдф, скрипалей, отравить..."


Теперь нужно добавить новый датасет к старому

In [91]:
cyrillic_pattern = re.compile(r'[а-яА-ЯёЁ]+')

def is_russian(text):
    return bool(cyrillic_pattern.search(str(text)))

russian_samples = porno_dataset[porno_dataset['title'].apply(is_russian)]
russian_samples

class_1_samples = russian_samples[russian_samples['label'] == 1].sample(n=7000, random_state=SEED)
class_0_samples = russian_samples[russian_samples['label'] == 0].sample(n=5000, random_state=SEED)

selected_samples = pd.concat([class_1_samples, class_0_samples])
selected_samples = selected_samples.rename(columns={'label': 'toxic', 'title': 'comment'})
selected_samples['cleared_text'] = selected_samples[['comment']].applymap(splitter)
selected_samples.head()

,comment,toxic,cleared_text
118960,мамка волосатый киска делать минуть секс,1,"[мамка, волосатый, киска, делать, минуть, секс]"
8546,русский телок сочный натуральный тело трахать ...,1,"[русский, телок, сочный, натуральный, тело, тр..."
41048,красивый попка чёрный горничная,1,"[красивый, попка, чёрный, горничная]"
25512,порно анал игрушка поп офицер анал больно подб...,1,"[порно, анал, игрушка, поп, офицер, анал, боль..."
52806,смазливый шимале очки мыть петух порно фото,1,"[смазливый, шимале, очки, мыть, петух, порно, ..."


Так, теперь надо слить два датасета в один

In [92]:
total_dataset = pd.concat([comments_dataset, selected_samples], axis=0)
total_dataset.head()

,comment,toxic,cleared_text
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"[верблюд, дебил, бл]"
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"[хохол, это, отдушина, затюканого, россиянин, ..."
2,Собаке - собачья смерть\n,1.0,"[собака, собачий, смерть]"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"[страница, обновить, дебил, это, оскорбление, ..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"[убедить, страничный, пдф, скрипалей, отравить..."


In [93]:
print(total_dataset.shape)

(27104, 3)


In [94]:
print(f"Количество токсичных (агрессивных) комментариев: {len(total_dataset[total_dataset['toxic'] == 1])}")
print(f"Количество нейтральных (адекватных) комментариев: {len(total_dataset[total_dataset['toxic'] == 0])}")

Количество токсичных (агрессивных) комментариев: 12518
Количество нейтральных (адекватных) комментариев: 14586


Примерно выровняли

Пробуем обучить Word2Vec

In [122]:
sentences = total_dataset['cleared_text']
w2v_model_new = Word2Vec(sentences, vector_size=768, window=5, min_count=1, sg = 0, seed = 42, epochs = 20)

In [123]:
w2v_model_new.wv.most_similar(positive='ум')

[('страдать', 0.9459784030914307),
 ('слабость', 0.9404314160346985),
 ('отъебнуть', 0.9381406307220459),
 ('жадность', 0.9341068863868713),
 ('примандюливаться', 0.9323843121528625),
 ('наоборот', 0.9319925904273987),
 ('затрахать', 0.9317011833190918),
 ('надристать', 0.9294644594192505),
 ('равнодушие', 0.9290173053741455),
 ('поддержать', 0.9279018044471741)]

Что-то релевантное получилось добыть

Сохранение word2vec

In [112]:
w2v_model_new.save("word2vec.model")

In [39]:
total_dataset.to_csv("../data/processed/total_dataset.csv")

In [125]:
X_vector = np.array([np.mean([w2v_model_new.wv[word] for word in sentence if word in w2v_model_new.wv] or [np.zeros(768)], axis=0) for sentence in sentences])
y = comments_dataset['toxic'].values

In [126]:
X_vector.shape

(27104, 768)

In [127]:
X, y = total_dataset['cleared_text'], total_dataset['toxic']
X_train, X_test, y_train, y_test = train_test_split(X_vector, 
                                                    y, 
                                                    test_size=0.2, 
                                                    stratify=y,
                                                    random_state=42)

In [128]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression()

In [129]:
print(accuracy_score(y_true=y_test, y_pred=log_reg.predict(X_test)))

0.8629404168972514


Это Word2Vec

Pipeline:

In [130]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


total_dataset = total_dataset.dropna(subset=['comment'])
X = total_dataset[['comment']].dropna()
y = total_dataset['toxic']

pipeline = Pipeline([
    (
        'features', 
        ColumnTransformer([
            (
                'comment',
                TfidfVectorizer(
                    lowercase=True, ngram_range=(4, 4), token_pattern=TOKEN_PATTERN,
                    stop_words=list(stopword_set), min_df=1, max_df=0.8, analyzer='char_wb', smooth_idf=True
                ), 
                'comment'  
            ),
        ])
    ),
    ('Logistic Regression', LogisticRegression(C = 5))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print(classification_report(y_test, y_pred))

Accuracy: 0.93
              precision    recall  f1-score   support

         0.0       0.91      0.97      0.94      2899
         1.0       0.96      0.88      0.92      2522

    accuracy                           0.93      5421
   macro avg       0.93      0.93      0.93      5421
weighted avg       0.93      0.93      0.93      5421



In [135]:
import joblib
joblib.dump(pipeline, 'toxic_comment_classifier.pkl')

['toxic_comment_classifier.pkl']